# GOLD - DASHBOARD LEVEL TABLES
# Purpose: CREATE 3 TABLES TO ANSWER BUSINESS QUESTION
# Source: DELTA TABLE
# Output: DELTA TABLES
## - gold.houses_per_quartile
## - gold.properties in each category
## - gold.changes in cateogry

## CONFIG/PARAMETERS

In [0]:
%sql
USE CATALOG harris_county_catalog

## TABLE CREATION

This table shows the amount of properties in each quartile

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.properties_in_quartile
SELECT 
  quartile
  , COUNT(*) as amount_of_properties_in_quartile
FROM harris_county_catalog.gold.cummulative_scd
WHERE end_date = '2025-01-01'
GROUP BY quartile

quartile,amount_of_properties_in_quartile
75%>,112433
50-75%,289658
25-50%,631303
<25%,121122


This table categorizes each house depending on how many years they have remained in the same quartile, then counts them

Categories:
- Very very stable (8 years in the same quartile)
- Very stable (4 years in the same quartile)
- Stable (2 years in the same quartile)
- Variable (less than 2 years in the same quartile)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.stability_categorization
with interval_value(
  SELECT
    *
    , CAST(end_date - start_date AS INT ) AS `interval`
  FROM harris_county_catalog.gold.cummulative_scd
)
, price_categorization_table
(
  SELECT 
    *
    , CASE 
        WHEN `interval` > 2920
          THEN 'Very very stable price'
        WHEN `interval` > 1460
          THEN 'Very stable price'
        WHEN `interval` > 730
          THEN 'stable price'
        ELSE 'variable price'
    END as stability_categorization
  FROM interval_value
  WHERE end_date = '2025-01-01'
-- AND `interval` > 2500
)
SELECT 
  stability_categorization
  , COUNT(CASE WHEN quartile = '<25%' THEN 1 END) AS q_0_25
  , COUNT(CASE WHEN quartile = '25-50%' THEN 1 END) AS q_25_50
  , COUNT(CASE WHEN quartile = '50-75%'THEN 1 END) AS q_50_75
  , COUNT(CASE WHEN quartile = '75%>' THEN 1 END) AS q_75_100
FROM price_categorization_table
GROUP BY stability_categorization
ORDER BY
  CASE stability_categorization
    WHEN 'Very very stable price' THEN 1
    WHEN 'Very stable price'      THEN 2
    WHEN 'Stable price'           THEN 3
    WHEN 'Variable price'         THEN 4
    ELSE 5
  END

price_categorization,q_0_25,q_25_50,q_50_75,q_75_100
Very very stable price,97554,146250,45262,46812
Very stable price,3303,205891,48729,16002
stable price,1693,196651,145573,32707
variable price,18572,82511,50094,16912


This table counts how many times a property has changed quartiles. Then counts how many properties have changed the same amount of times

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.amount_of_quartile_changes
WITH changes_in_quartiles AS (
  SELECT
    *
    , ROW_NUMBER() OVER (
        PARTITION BY dim_account_number
        ORDER BY end_date
      ) AS changes_in_quartiles
  FROM harris_county_catalog.gold.cummulative_scd
)
, maximums_counted AS (
  SELECT
    changes_in_quartiles - 1 AS quartile_changes
    , COUNT(CASE WHEN quartile = '<25%' THEN 1 END) AS q_0_25
    , COUNT(CASE WHEN quartile = '25-50%' THEN 1 END) AS q_25_50
    , COUNT(CASE WHEN quartile = '50-75%'THEN 1 END) AS q_50_75
    , COUNT(CASE WHEN quartile = '75%>' THEN 1 END) AS q_75_100
  FROM changes_in_quartiles
     WHERE end_date = '2025-01-01'
  GROUP BY changes_in_quartiles
)
SELECT *
FROM maximums_counted
ORDER BY quartile_changes

quartile_changes,q_0_25,q_25_50,q_50_75,q_75_100
0,104477,188568,76053,54048
1,380,381895,168457,42162
2,16075,45690,25781,9074
3,66,13906,17112,6026
4,119,1119,1932,816
5,4,113,270,273
6,1,11,50,31
7,0,1,3,3
